In [1]:
import os 
from model import AfrikaPOS

/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LANG = 'ewe'
DATA_DIR = os.path.join('..', 'data_source', 'masakhane-pos', 'data', LANG)
BASE_MODEL = 'bert-base-uncased'

NUM_HIDDENS = 2
NUM_ATTENTIONS = 1

In [3]:
model = AfrikaPOS(DATA_DIR, NUM_HIDDENS, NUM_ATTENTIONS, BASE_MODEL)




huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map: 100%|██████████| 145/145 [00:00<00:00, 17010.91 examples/s]


In [4]:
model.train_model(f"./models/{LANG}")

 50%|█████     | 46/92 [00:05<00:05,  8.08it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

{'eval_loss': 0.6639284491539001, 'eval_precision': 0.5738403022562069, 'eval_recall': 0.5907577019150708, 'eval_f1': 0.5361815832639302, 'eval_accuracy': 0.5907577019150708, 'eval_runtime': 0.5648, 'eval_samples_per_second': 256.706, 'eval_steps_per_second': 17.704, 'epoch': 1.0}


 99%|█████████▉| 91/92 [00:09<00:00, 14.73it/s]/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/bapa/miniconda3/envs/afrika-pos/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

{'eval_loss': 0.5893334746360779, 'eval_precision': 0.6131046162908795, 'eval_recall': 0.6298917568692756, 'eval_f1': 0.5853055015698655, 'eval_accuracy': 0.6298917568692756, 'eval_runtime': 0.1638, 'eval_samples_per_second': 885.156, 'eval_steps_per_second': 61.045, 'epoch': 2.0}


100%|██████████| 92/92 [00:10<00:00,  8.70it/s]

{'train_runtime': 10.5764, 'train_samples_per_second': 137.666, 'train_steps_per_second': 8.699, 'train_loss': 0.8968824303668478, 'epoch': 2.0}
